# Inital processing of CERT dataset

Output of this notebook is folder with processed `.csv` files with all features needed and with much smaller filesize

In [ ]:
import pandas as pd
from pathlib import Path
import re
from tqdm import tqdm
from math import ceil
import glob
import gc

tqdm.pandas()

In [ ]:
dataset_version = '6.2'

answers_dir = Path(r"C:\Datasets\CERT\answers")
dataset_dir = Path(rf"C:\Datasets\CERT\r{dataset_version}")

main_answers_file = answers_dir / "insiders.csv"

assert(answers_dir.is_dir())
assert(dataset_dir.is_dir())
assert(main_answers_file.is_file())

output_dir = Path(f'C:/Datasets/CERT_output_v{dataset_version}/')
assert(output_dir.is_dir())

In [ ]:
main_df = pd.read_csv(main_answers_file)
main_df = main_df[main_df['dataset'] == dataset_version]

malicious_users = main_df.user.unique()

In [ ]:
insider_files = list(answers_dir.glob(f'**/r{dataset_version}*.csv'))

df_ls = []

for filename in insider_files:
    df = pd.read_csv(filename, names=list(range(13)))
    df_ls.append(df)
    
df = pd.concat(df_ls, axis=0, ignore_index=True)

Сайты, которые посещают инсайдеры:

In [ ]:
set(df[df[0] == 'http'][5].apply(lambda s: re.match('^https?://(www\.)?([0-9\-\w\.]+)?.+$', s).group(2)))

In [ ]:
device_cols = next(pd.read_csv(dataset_dir / 'device.csv', chunksize=10)).columns
email_cols = next(pd.read_csv(dataset_dir / 'email.csv', chunksize=10)).columns
file_cols = next(pd.read_csv(dataset_dir / 'file.csv', chunksize=10)).columns
http_cols = next(pd.read_csv(dataset_dir / 'http.csv', chunksize=10)).columns
logon_cols = next(pd.read_csv(dataset_dir / 'logon.csv', chunksize=10)).columns

# Читаем датасет

## Logon

In [ ]:
df = pd.read_csv(dataset_dir / 'logon.csv')

In [ ]:
df['date'] = pd.to_datetime(df.date, format='%m/%d/%Y %H:%M:%S')

In [ ]:
df['day'] = df['date'].dt.floor('D')

# replaces df.groupby(['user', 'day']).pc.agg(pd.Series.mode)
# it much more complicated but much, much faster
# credit to: https://stackoverflow.com/a/57179083

# result is exactly same as "df.groupby('user').pc.agg(lambda x:x.value_counts().index[0])" though

most_common_pc = df\
    .groupby(['user', 'day', 'pc'])\
    .size()\
    .to_frame('count')\
    .reset_index()\
    .sort_values('count', ascending=False)\
    .drop_duplicates(subset=['user', 'day'])\
    .drop(columns=['count'])\
    .sort_values(['user', 'day'])\
    .groupby('user')\
    .pc\
    .agg(pd.Series.mode)\
    .rename('most_common_pc')
most_common_pc

In [ ]:
df = df.merge(most_common_pc, left_on='user', right_on='user', )
df['is_usual_pc'] = df['most_common_pc'] == df['pc']

is_work_time = (8 <= df.date.dt.hour) & (df.date.dt.hour < 17)
df['is_work_time'] = is_work_time

df['subtype'] = df['activity']

In [ ]:
df[['date', 'user', 'is_usual_pc', 'is_work_time', 'subtype']].to_csv(output_dir / 'logon_preprocessed.csv')

## Device

In [ ]:
csv_name = 'device'

df = pd.read_csv(dataset_dir / f'{csv_name}.csv')
df

In [ ]:
df['date'] = pd.to_datetime(df.date, format='%m/%d/%Y %H:%M:%S')

df = df.merge(most_common_pc, left_on='user', right_on='user', )
df['is_usual_pc'] = df['most_common_pc'] == df['pc']

is_work_time = (8 <= df.date.dt.hour) & (df.date.dt.hour < 17)
df['is_work_time'] = is_work_time

df['subtype'] = df['activity']

In [ ]:
df[['date', 'user', 'is_usual_pc', 'is_work_time', 'subtype']].to_csv(output_dir / f'{csv_name}_preprocessed.csv')

## File

In [ ]:
csv_name = 'file'

df = pd.read_csv(dataset_dir / f'{csv_name}.csv', usecols=['date', 'user', 'pc', 'filename'])
df

In [ ]:
df['date'] = pd.to_datetime(df.date, format='%m/%d/%Y %H:%M:%S')

df = df.merge(most_common_pc, left_on='user', right_on='user', )
df['is_usual_pc'] = df['most_common_pc'] == df['pc']

is_work_time = (8 <= df.date.dt.hour) & (df.date.dt.hour < 17)
df['is_work_time'] = is_work_time

file_extensions = df.filename.str[-4:]
df['subtype'] = file_extensions

In [ ]:
df

In [ ]:
df[['date', 'user', 'is_usual_pc', 'is_work_time', 'subtype']].to_csv(output_dir / f'{csv_name}_preprocessed.csv')

## Email

In [ ]:
csv_name = 'email'

df = pd.read_csv(dataset_dir / f'{csv_name}.csv', usecols=['date', 'user', 'pc', 'to', 'cc', 'bcc', 'from'])
df = df.fillna('')
df

In [ ]:
from_domain = df['from'].str.extract('^.+@(.+$)')[0]
is_external_from = from_domain == 'dtaa.com'

In [ ]:
# this lines takes ~10 mins
# to_concated = df[['to', 'cc', 'bcc']].agg(';'.join, axis=1)

# slighly slower but there is nice progress bar
to_concated = df[['to', 'cc', 'bcc']].progress_apply(lambda x: ';'.join([x.to, x.cc, x.bcc]), axis=1)

In [ ]:
# yes, it's horrible but this part is somewhat fast compared to the join part

is_external_to = to_concated.progress_apply(
    lambda x: any([re.match('^.+@(.+$)', e).group(1) != 'dtaa.com' for e in x.split(';') if e != ''])
    )

In [ ]:
is_external = is_external_to | is_external_to

In [ ]:
df['date'] = pd.to_datetime(df.date, format='%m/%d/%Y %H:%M:%S')

df = df.merge(most_common_pc, left_on='user', right_on='user', )
df['is_usual_pc'] = df['most_common_pc'] == df['pc']

is_work_time = (8 <= df.date.dt.hour) & (df.date.dt.hour < 17)
df['is_work_time'] = is_work_time

df['subtype'] = is_external

In [ ]:
df[['date', 'user', 'is_usual_pc', 'is_work_time', 'subtype']].to_csv(output_dir / f'{csv_name}_preprocessed.csv')

## HTTP

In [ ]:
job_hunting_websites = [
    'careerbuilder.com',
    'craiglist.org',
    'indeed.com',
    'job-hunt.org',
    'jobhuntersbible.com',
    'linkedin.com',
    'monster.com',
    'simplyhired.com',
]

hacktivist_websites = [
    'actualkeylogger.com',
    'best-spy-soft.com',
    'dailykeylogger.com',
    'keylogpc.com',
    'refog.com',
    'relytec.com',
    'softactivity.com',
    'spectorsoft.com',
    'webwatchernow.com',
    'wellresearchedreviews.com',
    'wikileaks.org'
]

filesharing_websites = [
    '4shared.com'
    'dropbox.com',
    'fileserve.com',
    'filefreak.com',
    'filestube.com',
    'megaupload.com',
    'thepiratebay.org'
]

In [ ]:
with open(dataset_dir / 'http.csv') as f:
    for count, l in tqdm(enumerate(f)):
        pass
count

In [ ]:
# fast substitute for previous cell
# count = 28434424

In [ ]:
CHUNK_SIZE = 500000

df_iter = pd.read_csv(dataset_dir / 'http.csv', chunksize=CHUNK_SIZE, usecols=['date', 'user', 'pc', 'url'])
# (output_dir / 'http_preprocessed.csv').unlink()
first_it = True
mode = 'w'

for http_df in tqdm(df_iter, total=ceil(count / CHUNK_SIZE)):
    http_df['date'] = pd.to_datetime(http_df.date, format='%m/%d/%Y %H:%M:%S')

    site_names = http_df['url'].apply(lambda s: re.match('^https?://(www)?([0-9\-\w\.]+)?.+$', s).group(2))
    http_df['site_name'] = site_names
    
    http_df['subtype'] = 0
    http_df.loc[site_names.isin(job_hunting_websites), 'subtype'] = 1
    http_df.loc[site_names.isin(hacktivist_websites), 'subtype'] = 2
    http_df.loc[site_names.isin(filesharing_websites), 'subtype'] = 3
    
    http_df = http_df.merge(most_common_pc, left_on='user', right_on='user', )
    http_df['is_usual_pc'] = http_df['most_common_pc'] == http_df['pc']

    is_work_time = (8 <= http_df.date.dt.hour) & (http_df.date.dt.hour < 17)
    http_df['is_work_time'] = is_work_time
    
    http_df.to_csv(output_dir / 'http_preprocessed.csv', header=first_it, index=False,
                   mode=mode, columns=['date', 'user', 'is_usual_pc', 'is_work_time', 'subtype', 'site_name'])
    first_it = False
    mode = 'a'

# LDAP data

In [ ]:
LDAP_dir = dataset_dir / 'LDAP'
assert LDAP_dir.is_dir()
ldap_columns = ['role', 'business_unit', 'functional_unit', 'department', 'team']

In [ ]:
# ldap_df_list = []

# for file_path in LDAP_dir.glob('*.csv'):
#     year, month = file_path.stem.split('-')
#     ldap_df = pd.read_csv(file_path)
    

#     ldap_df = ldap_df[ldap_columns + ['user_id']]
#     ldap_df['year'] = int(year)
#     ldap_df['month'] = int(month)
    
#     ldap_df_list.append(ldap_df)
    
# ldap_df = pd.concat(ldap_df_list)
# del ldap_df_list
# gc.collect()

In [ ]:
users = ldap_df.user_id.unique()
for user in users:
    user_df = ldap_df[ldap_df['user_id'] == user]
    for col in ldap_columns:
        assert user_df[col].unique().shape[0] == 1

This means users in CERT 4.2 do not change their units, departments, etc.

In [ ]:
ldap_df = pd.read_csv(LDAP_dir / '2009-12.csv')
for col in ldap_columns:
    ldap_df[col] = ldap_df[col].astype('category')
ldap_df = ldap_df[['user_id'] + ldap_columns]

ldap_df.to_csv(output_dir / 'LDAP.csv', index=False)